Code modified from Matthew A. Russell's book "Mining the Social Web: Data Mining Facebook, Twitter, LinkedIn, Google+, GitHub, and More"

In [26]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%qtconsole --colors=Linux

In [21]:
import twitter
import json
import pandas as pd

# Set up API to my personal twitter account via Twitter's Python module

CONSUMER_KEY = "SixboXZyf3ODcR0K2vdAjj7nh"
CONSUMER_SECRET = "RoOFdhL7OO2sDMIYwbAh1IJnm2r0qA3n9iQFTj4owpUVutvWOd"
OAUTH_TOKEN = "36209712-WnGG44AXJ9xWHP7jtmTWPyInxrETh20uY7OqJnsvE"
OAUTH_TOKEN_SECRET = "cmKF3sY3fuvfzialocuXcPbw0DDlx84L7gBKQnGrE0t6r"
auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth = auth)


'\nhashtag_search = "#JeSuisParis"\ncount = 300\n\nsearch_res = twitter_api.search.tweets(q=hashtag_search,count = count)\nstatuses = search_res[\'statuses\']\n#\t\tIterate through 5 more batches of results by following the cursor\n\nfor _ in range(5):\n\tprint("Length of statuses",len(statuses))\n\ttry: \n\t\tnext_res = search_res[\'search_metadata\'][\'next_res\']\n\texcept KeyError as e: #\t\tNo more results when results don\'t exist\n\t\tbreak\n\t#\t\tCreate a dict from next_res\n\tkwargs = dict([kv.split(\'=\') for kv in next_res[1:].split(\'&\')])\n\tsearch_res = twitter_api.search.tweets(**kwargs)\n\tstatuses+=search_res[\'statuses\']\n\n#print(json.dumps(statuses[0],indent =1))\n\n'

In [134]:
# This function accepts my api, search query, and max number of results and returns most recent dict of twitter data
def twitter_search(twitter_api, q, max_results=1000):
    search_results = twitter_api.search.tweets(q = q, count = 200)
    statuses = search_results['statuses']
    max_results = min(1000,max_results)
    for _ in range(10):          
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: 
            break
        kwargs = dict([ keyvalue.split('=') for keyvalue in next_results[1:].split("&")])
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        if len(statuses) > max_results:
            break
    return statuses
    

In [135]:
q = '#JeSuisParis'
statuses = twitter_search(twitter.Twitter(auth = auth),q)

"tweets_data = pd.DataFrame({'favorites_count' : favorites,\n 'retweet_count' : retweets,\n 'media_attachment':media\n  })\n"

In [137]:
import numpy ase np
import re
# Create empty data frame to write with the twitter data
df = pd.DataFrame({'status' : statuses})
df['favorites'] = np.nan
df['retweets'] = np.nan
df['media'] = np.nan

# For each status in statuses, grab favorites, retweets, and media entities
for j in range(len(statuses)):                           
    favorite = statuses[j]['favorite_count']
    retweet = statuses[j]['retweet_count'] 
    entities = statuses[j]['entities']
    # Match string 'media' to locate media dictionary in each tweet's data
    entities = str(entities)
    match = re.search(r'media',entities)
    # If matched, locate 'type' key and replace media list entry with its associated value and remove extra notation
    if match:
            entities = entities.split('type')[1]
            entities = entities[4:]
            entities = entities.replace("'", "-")
            df.loc[j, 'media'] = entities.split('-')[0]
    # rewrite the data frame
    df.loc[j, 'favorites'] = favorite
    df.loc[j, 'retweets'] = retweet
# replace missing values in media column with "none" for no media attachment
df['media'].fillna('None', inplace=True)

# create finished data frame
df = df[['favorites','retweets','media']]

In [138]:
# write to csv- analysis continues in R
df.to_csv("twitter.csv")